In [1]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [2]:
# =============================================
# THIS comes from INTERFACE
#
selected_exposure = '00000003'# args['exposure']
selected_arm = 'r' # args['arm']
selected_spectrograph =  4 # args['spectrograph']

# =============================================
# THIS comes from QLF.CFG
#
night = '20190101'

# ============================================
#  THIS READ yaml files
#
from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp = selected_exposure # intentionaly redundant
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

# =============================================
# THIS is only to simplify the code understanding
#
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

countpix loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countpix-r4-00000003.yaml
getbias loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getbias-r4-00000003.yaml
getrms loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getrms-r4-00000003.yaml
xwsigma loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-xwsigma-r4-00000003.yaml
countbins loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countbins-r4-00000003.yaml
integ loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-integ-r4-00000003.yaml
skycont loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skycont-r4-00000003.yaml
skypeak loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skypeak-r4-00000003.yaml
skyresid loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skyresid-r4-00000003.yaml

In [3]:
def palette(name_of_mpl_palette):
    """ Transforms a matplotlib palettes into a bokeh 
    palettes
    """
    from matplotlib.colors import rgb2hex
    import matplotlib.cm as cm
    colormap =cm.get_cmap(name_of_mpl_palette) #choose any matplotlib colormap here
    bokehpalette = [rgb2hex(m) for m in colormap(np.arange(colormap.N))]
    return bokehpalette

#nipy_spectral)#Viridis256)#,RdYlBu11)#Viridis256)#, low=0, high=100)
my_palette = palette("viridis")

In [6]:
badf_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">RA: </span>
            <span style="font-size: 13px; color: #515151;">@ra</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">DEC: </span>
            <span style="font-size: 13px; color: #515151;">@dec</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">FIBER ID: </span>
            <span style="font-size: 13px; color: #515151;">@fiberid</span>
        </div>
    </div>
"""
c1,c2 = selected_spectrograph*500, (selected_spectrograph+1)*500
qlf_fiberid = np.arange(0,5000)[c1:c2] 

hover= HoverTool(tooltips=badf_tooltip)


p = figure( title = 'SKY_FIBERS', x_axis_label='RA', y_axis_label='DEC'
           , plot_width=600, plot_height=600
           , tools= [hover, "pan,box_zoom,reset,crosshair"])
skyfibers = skycont['SKYFIBERID']
col = []

for x in range(500):
    if x in skyfibers:
        col.append('sky')
    else:
        col.append('good')



from bokeh.models import CategoricalColorMapper

# Convert df to a ColumnDataSource: source
source = ColumnDataSource(data={
            'ra' : skycont['RA'][c1:c2],
            'dec': skycont['DEC'][c1:c2],
            'bad': col,
            'fiberid': qlf_fiberid
            })

# Make a CategoricalColorMapper object: color_mapper
color_mapper = CategoricalColorMapper(factors=['good', 'sky'],
                                      palette=['lightgrey', 'blue'])

# Add a circle glyph to the figure p
p.circle('ra', 'dec', source=source
         , radius = 0.015
         , color=dict(field='bad', transform=color_mapper)
         , line_color='black', line_width=0.6
         , fill_alpha=0.8
           )#, legend='origin')
output_notebook()
show(p)

Loading BokehJS ...

### OUTRO:

cabe: curdoc para trocar SKY FIBERS e BADFIBERS

In [9]:
skycont.keys()

dict_keys(['RA', 'SKYCONT_AMP', 'DEC', 'SKYCONT_ERR', 'SKYFIBERID', 'SKYCONT_FIBER', 'SKYCONT'])

5000